In [0]:
%run ./conf_notebook

2024-05-08 16:06:58.921672: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 16:06:58.925730: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 16:06:58.985274: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 16:07:00.063877: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


path,name,size,modificationTime
dbfs:/mnt/pqsashrd/data/ann_pred_1.csv/,ann_pred_1.csv/,0,0
dbfs:/mnt/pqsashrd/data/ann_pred_1_test.csv/,ann_pred_1_test.csv/,0,0
dbfs:/mnt/pqsashrd/data/ann_pred_3.csv/,ann_pred_3.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_1.csv/,clust_pred_1.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_1_test.csv/,clust_pred_1_test.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_3.csv/,clust_pred_3.csv/,0,0
dbfs:/mnt/pqsashrd/data/clust_pred_3_test.csv/,clust_pred_3_test.csv/,0,0
dbfs:/mnt/pqsashrd/data/test_1.csv,test_1.csv,38574669,1714569841000
dbfs:/mnt/pqsashrd/data/test_2.csv,test_2.csv,35056701,1714569835000
dbfs:/mnt/pqsashrd/data/test_3.csv,test_3.csv,25575183,1714569796000


Import the data

In [0]:
df_train = ps.read_csv(train_3_csv_file_path)
df_train = df_train.drop('_c0', axis=1)

df_test = ps.read_csv(test_3_csv_file_path)
df_test = df_test.drop('_c0', axis=1)

In [0]:
label_encoder = LabelEncoder()

df_train = df_train.to_pandas()
df_test = df_test.to_pandas()

df_train["attack_cat_encoded"] = label_encoder.fit_transform(df_train["attack_cat"])
df_test["attack_cat_encoded"] = label_encoder.transform(df_test["attack_cat"])

df_train = ps.DataFrame(df_train)
df_test = ps.DataFrame(df_test)

In [0]:
X_train = df_train.set_index("attack_cat")
X_train = X_train.drop("attack_cat_encoded", axis=1)
y_train = df_train["attack_cat_encoded"]

X_test = df_test.set_index("attack_cat")
X_test = X_test.drop("attack_cat_encoded", axis=1)
y_test = df_test["attack_cat_encoded"]

%md
Adjusted Rand Index (ARI): This metric measures the similarity between two clusterings by considering all pairs of samples and counting pairs that are assigned in the same or different clusters in the predicted and true clusterings. The ARI is adjusted for chance, which makes it more robust for comparing different clustering algorithms.

Silhouette Coefficient: This metric measures how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The Silhouette Coefficient is calculated for each sample and can range from -1 to +1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters. If most objects have a high value, the clustering configuration is appropriate.

### Fuzzy K-MEANS 

In [0]:
estimator = LDA(n_components = 2)
X_lda = estimator.fit_transform(ps.DataFrame(X_train).to_pandas(), y_train.to_pandas())
X_lda_df = ps.DataFrame(data=X_lda, columns=['LDA_component_1', 'LDA_component_2'], index=ps.DataFrame(X_train).index)
X_lda_df = X_lda_df.to_pandas()

X_lda_test = estimator.transform(ps.DataFrame(X_test).to_pandas())
X_lda_df_test_ = ps.DataFrame(data=X_lda_test, columns=['LDA_component_1', 'LDA_component_2'], index=ps.DataFrame(X_test).index)
X_lda_df_test_ = X_lda_df_test_.to_pandas()

In [0]:
custom_color_scale = [
    '#800080',  # Purple (no change)
    '#9370db',  # Medium Purple (no change)
    '#6495ed',  # Cornflower Blue (no change)
    '#000080',  # Navy (no change)
    '#6998b3',  # Darker Light Blue (no change)
    "#e4cbd1",  # Medium Violet Red (a darker, more subdued pink)
]

In [0]:
estimator = LDA(n_components = 4)
X_lda = estimator.fit_transform(X_train.to_pandas(), y_train.to_pandas())
X_lda_df_ = ps.DataFrame(data=X_lda, columns=['LDA_component_1', 'LDA_component_2', 'LDA_component_3', 'LDA_component_4'], index=X_train.index)
X_lda_df_ = X_lda_df_.to_pandas()

X_lda_test = estimator.transform(X_test.to_pandas())
X_lda_test_df = ps.DataFrame(data=X_lda_test, columns=['LDA_component_1', 'LDA_component_2', 'LDA_component_3', 'LDA_component_4'], index=X_test.index)
X_lda_test_df = X_lda_test_df.to_pandas()  

In [0]:
k=5
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(X_lda_df_.T, c=k, m=2, error=0.00001, maxiter=100000)
predicted_labels = np.argmax(u, axis=0)

true_labels = y_train.to_numpy()
ari = metrics.adjusted_rand_score(true_labels, predicted_labels)

spark_df = spark.createDataFrame([(Vectors.dense(x), int(l)) for x, l in zip(X_lda_df_.values, predicted_labels)], ["features", "prediction"])
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='features', metricName='silhouette', distanceMeasure='squaredEuclidean')
silhouette_score = evaluator.evaluate(spark_df)

print(f"Number of clusters = {k}: Silhouette Coefficient = {silhouette_score}, Adjusted Rand Index (ARI) = {ari}")

Number of clusters = 5: Silhouette Coefficient = 0.8649812088089857, Adjusted Rand Index (ARI) = 0.5502212008666248


In [0]:
u_test, u0_test, d_test, jm_test, p_test, fpc_test = cmeans_predict(X_lda_test_df.T, cntr, m=5, error=0.00001, maxiter=100000)
predicted_labels_test = np.argmax(u_test, axis=0)

true_labels = y_test.to_numpy()
ari = metrics.adjusted_rand_score(true_labels, predicted_labels_test)

spark_df = spark.createDataFrame([(Vectors.dense(x), int(l)) for x, l in zip(X_lda_test_df.values, predicted_labels_test)], ["features", "prediction"])
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='features', metricName='silhouette', distanceMeasure='squaredEuclidean')
silhouette_score = evaluator.evaluate(spark_df)

print(f"Number of clusters = {k}: Silhouette Coefficient = {silhouette_score}, Adjusted Rand Index (ARI) = {ari}")

Number of clusters = 5: Silhouette Coefficient = 0.8688813108656244, Adjusted Rand Index (ARI) = 0.43052079646434704


In [0]:
def map_predicted_labels(num):
    if num == 0:
        return 2
    elif num == 1:
        return 4
    elif num == 2:
        return 0
    elif num == 3:
        return 1
    elif num == 4:
        return 3
    else:
        return num

predicted_labels_transformed = list(map(map_predicted_labels, predicted_labels))
predicted_labels_transformed_test = list(map(map_predicted_labels, predicted_labels_test))

In [0]:
X_lda_df_ = X_lda_df_.reset_index()
X_lda_df_["clustering_pred"] = predicted_labels_transformed
soft_classification = pd.DataFrame(u.T, columns=['clustering_prob_0', 'clustering_prob_1', 'clustering_prob_2', 'clustering_prob_3', 'clustering_prob_4'])
X_lda_df_ = pd.concat([X_lda_df_, soft_classification], axis=1)

X_lda_test_df = X_lda_test_df.reset_index()
X_lda_test_df["clustering_pred"] = predicted_labels_transformed_test
soft_classification_test = pd.DataFrame(u_test.T, columns=['clustering_prob_0', 'clustering_prob_1', 'clustering_prob_2', 'clustering_prob_3', 'clustering_prob_4'])
X_lda_test_df = pd.concat([X_lda_test_df, soft_classification_test], axis=1)

In [0]:
X_lda_df_ = ps.DataFrame(X_lda_df_).to_spark()
save_path = clust_pred_3_csv_file_path
X_lda_df_.coalesce(1).write.mode("overwrite").option("header", "true").csv(save_path)

In [0]:
X_lda_test_df = ps.DataFrame(X_lda_test_df).to_spark()
save_path = clust_pred_3_test_csv_file_path
X_lda_test_df.coalesce(1).write.mode("overwrite").option("header", "true").csv(save_path)